In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Serving-meta-llama/Llama-3.1-8B-Instruct-TGI-GCS-Vertex AI-L4
> [**Deploy Meta Llama 3.1 405B on Google Cloud Vertex AI** ](https://github.com/huggingface/blog/blob/main/llama31-on-vertex-ai.md)

> [**Text Generation Inference (TGI)**](https://github.com/huggingface/text-generation-inference) is a toolkit developed by Hugging Face for deploying and serving LLMs, with high performance text generation.

> [**Hugging Face DLCs**](https://github.com/huggingface/Google-Cloud-Containers) are pre-built and optimized Deep Learning Containers (DLCs) maintained by Hugging Face and Google Cloud teams to simplify environment configuration for your ML workloads.

In [1]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform huggingface_hub

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# @title Define project information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [3]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)


In [4]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  y


Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


## Requirements
- Artifact Registry Reader (roles/artifactregistry.reader)
- Vertex AI User (roles/aiplatform.user)

In [5]:
# @title Enable APIs
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable artifactregistry.googleapis.com

## Register model on Vertex AI

In [6]:
from google.cloud import aiplatform
from huggingface_hub import get_token

model = aiplatform.Model.upload(
    display_name="Llama-3.1-8B-Instruct",
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310",
    
    # https://huggingface.co/docs/text-generation-inference/en/reference/launcher
    
    serving_container_environment_variables={
        "MODEL_ID": "meta-llama/Llama-3.1-8B-Instruct",
        "NUM_SHARD": "1",
        "MAX_INPUT_TOKENS": "512",
        "MAX_TOTAL_TOKENS": "1024",
        "MAX_BATCH_PREFILL_TOKENS": "1512",
        "HUGGING_FACE_HUB_TOKEN": get_token(),
    },
    serving_container_ports=[8080],
)
model.wait()

Creating Model
Create Model backing LRO: projects/721521243942/locations/us-central1/models/885041989173641216/operations/2606112934610337792
Model created. Resource name: projects/721521243942/locations/us-central1/models/885041989173641216@1
To use this Model in another session:
model = aiplatform.Model('projects/721521243942/locations/us-central1/models/885041989173641216@1')


## Deploy model on Vertex AI

In [7]:
deployed_model = model.deploy(
    endpoint=aiplatform.Endpoint.create(display_name="Llama-3.1-8B-Instruct"),
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

Creating Endpoint
Create Endpoint backing LRO: projects/721521243942/locations/us-central1/endpoints/1555774868442578944/operations/289855351258349568
Endpoint created. Resource name: projects/721521243942/locations/us-central1/endpoints/1555774868442578944
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/721521243942/locations/us-central1/endpoints/1555774868442578944')
Deploying model to Endpoint : projects/721521243942/locations/us-central1/endpoints/1555774868442578944
Deploy Endpoint model backing LRO: projects/721521243942/locations/us-central1/endpoints/1555774868442578944/operations/2595698360472043520
Endpoint model deployed. Resource name: projects/721521243942/locations/us-central1/endpoints/1555774868442578944


## Online predictions on Vertex AI

In [ ]:
%pip install --upgrade --user --quiet transformers jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 5.0 MB/s eta 0:00:00


After the installation is complete, the following snippet will apply the chat template to the conversation:

In [ ]:
import os
from huggingface_hub import get_token
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    token=get_token(),
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:

messages = [
    {"role": "system", "content": "You are an assistant that responds as an AI expert."},
    {"role": "user", "content": "What's Deep learning?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

output = deployed_model.predict(
    instances=[
        {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ]
)

output


#### From a different session

To run the online prediction from a different session, you can run the following snippet.

In [ ]:
import os

from google.cloud import aiplatform


aiplatform.init(project=PROJECT_ID, location=LOCATION)

endpoint_display_name = (
    "Llama-3.1-8B-Instruct"  # TODO: change to your endpoint display name
)


# Iterates over all the Vertex AI Endpoints within the current project and keeps the first match (if any), otherwise set to None
ENDPOINT_ID = next(
    (
        endpoint.name
        for endpoint in aiplatform.Endpoint.list()
        if endpoint.display_name == endpoint_display_name
    ),
    None,
)
assert ENDPOINT_ID, (
    "`ENDPOINT_ID` is not set, please make sure that the `endpoint_display_name` is correct at "
    f"https://console.cloud.google.com/vertex-ai/online-prediction/endpoints?project={os.getenv('PROJECT_ID')}"
)

endpoint = aiplatform.Endpoint(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
)


output = endpoint.predict(
    instances=[
        {
            "inputs": "<bos><start_of_turn>user\nWhat's Deep Learning? in details<end_of_turn>\n<start_of_turn>model\n",
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ],
)
output

Prediction(predictions=['Deep Learning is a subfield of Machine Learning that deals with Artificial Neural Networks (ANNs) that are composed of multiple layers. This multi-layered structure allows them to learn and represent complex data representations, enabling tasks like image and speech recognition, natural language processing, and playing complex games.\n\nHere\'s a detailed breakdown:\n\n1.  **Artificial Neural Networks (ANNs)**: ANNs are composed of interconnected nodes or "neurons" that process and transmit information. These nodes receive inputs, perform calculations, and send outputs to the next layer.\n2.  **Neural Layers**: ANNs are organized into layers, each with'], deployed_model_id='6301157102761017344', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/704449403334688768', explanations=None)

## Cleaning up

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

## References

- [GitHub Repository - Hugging Face DLCs for Google Cloud](https://github.com/huggingface/Google-Cloud-Containers): contains all the containers developed by the collaboration of both Hugging Face and Google Cloud teams; as well as a lot of examples on both training and inference, covering both CPU and GPU, as well support for most of the models within the Hugging Face Hub.
- [Google Cloud Documentation - Hugging Face DLCs](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face): contains a table with the latest released Hugging Face DLCs on Google Cloud.
- [Google Artifact Registry - Hugging Face DLCs](https://console.cloud.google.com/artifacts/docker/deeplearning-platform-release/us/gcr.io): contains all the DLCs released by Google Cloud that can be used.
- [Hugging Face Documentation - Google Cloud](https://huggingface.co/docs/google-cloud): contains the official Hugging Face documentation for the Google Cloud DLCs.